# Behavioural cloning project

In [1]:
# Configure matlab to show graphics in the notebook
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
# Change to 'data' when training on a GPU
PATH_TO_DATA = 'data/sample'

In [3]:
# Shape of the image
input_shape = (160, 320, 3)

In [4]:
def img_folder(data_folder):
    return '{}/IMG'.format(data_folder)

def path_driving_log(data_folder):
    return '{}/driving_log.csv'.format(data_folder)

In [5]:
datasets = ['train', 'test', 'valid']

In [6]:
def get_driving_log_dataframe(data_folder):
    driving_log_df = pd.read_csv(path_driving_log(data_folder))
    return driving_log_df

In [7]:
path_to_folders = dict(zip(datasets, map(lambda folder: '{0}/{1}'.format(PATH_TO_DATA, folder), datasets)))

In [8]:
path_to_folders

{'test': 'data/sample/test',
 'train': 'data/sample/train',
 'valid': 'data/sample/valid'}

In [11]:
from scipy.ndimage import imread
from os import listdir
from sklearn.utils import shuffle

def image_label_generator(data_folder, batch_size=64):
    driving_log_df = get_driving_log_dataframe(data_folder)
    number_of_examples = len(driving_log_df)
    image_columns = ['center', 'left', 'right']
    
    X_train = []
    y_train = []
    weights = []
    index_in_batch = 0
    batch_number = 0
    
    while True:
        for image_column in image_columns:
            image_series = driving_log_df[image_column]
            steering_series = driving_log_df['steering']
            for offset in range(0, number_of_examples, batch_size):
                X_train = []
                y_train = []
                weights = []

                end_of_batch = min(number_of_examples, offset + batch_size)

                for j in range(offset, end_of_batch):
                    image_filename = image_series[j].lstrip().rstrip()
                    image = imread('{0}/{1}'.format(data_folder, image_filename))
                    label = steering_series[j]
                    
                    if abs(label) < 0.2:
                        weights.append(1)
                    else:
                        weights.append(100)
                    
                    X_train.append(image)
                    y_train.append(label)
                    X_train, y_train, weights = shuffle(X_train, y_train, weights)

                yield np.array(X_train), np.array(y_train), np.array(weights)

In [29]:
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense
from keras.layers import Convolution2D
from keras.layers import Flatten

In [30]:
model = Sequential()

In [31]:
model.add(Convolution2D(3, 1, 1, 
                          input_shape=input_shape, 
                          border_mode='same', 
                          activation='relu'))
model.add(Convolution2D(24, 5, 5,
                       activation='relu',
                       subsample=(2, 2)))
model.add(Convolution2D(36, 5, 5,
                       activation='relu',
                       subsample=(2, 2)))
model.add(Convolution2D(48, 5, 5,
                       activation='relu',
                       subsample=(2, 2)))
model.add(Convolution2D(64, 3, 3,
                       activation='relu'))
model.add(Convolution2D(80, 3, 3,
                       activation='relu'))
model.add(Convolution2D(96, 3, 3,
                       activation='relu'))
model.add(Convolution2D(112, 3, 3,
                       activation='relu'))
model.add(Convolution2D(128, 3, 3,
                       activation='relu'))
model.add(Convolution2D(144, 3, 3,
                       activation='relu'))
model.add(Convolution2D(160, 3, 3,
                       activation='relu'))
model.add(Convolution2D(176, 3, 3,
                       activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))

In [32]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_14 (Convolution2D) (None, 160, 320, 3)   12          convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
convolution2d_15 (Convolution2D) (None, 78, 158, 24)   1824        convolution2d_14[0][0]           
____________________________________________________________________________________________________
convolution2d_16 (Convolution2D) (None, 37, 77, 36)    21636       convolution2d_15[0][0]           
____________________________________________________________________________________________________
convolution2d_17 (Convolution2D) (None, 17, 37, 48)    43248       convolution2d_16[0][0]           
___________________________________________________________________________________________

In [33]:
from keras.callbacks import ModelCheckpoint
filepath="weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [34]:
from keras.optimizers import Adam
adam = Adam(lr=0.0001)

In [35]:
model.compile(optimizer=adam, 
              loss='mse',
             metrics=['mean_squared_error'])

In [36]:
train_generator = image_label_generator(path_to_folders['train'])
valid_generator = image_label_generator(path_to_folders['valid'])

In [37]:
number_of_training_examples = len(get_driving_log_dataframe(path_to_folders['train']) * 3)
number_of_validation_examples = len(get_driving_log_dataframe(path_to_folders['valid']) * 3)

In [38]:
model.fit_generator(train_generator, 
                    samples_per_epoch=number_of_training_examples, 
                    nb_epoch=10, 
                    validation_data=valid_generator,
                    nb_val_samples=number_of_validation_examples,
                   callbacks=callbacks_list)

Epoch 1/10
Epoch 00000: loss improved from inf to 0.01025, saving model to weights-improvement-00-0.00.hdf5
12/12 [==============================] - 3s - loss: 0.0103 - mean_squared_error: 0.0103 - val_loss: 0.0029 - val_mean_squared_error: 0.0029
Epoch 2/10
Epoch 00001: loss improved from 0.01025 to 0.01015, saving model to weights-improvement-01-0.00.hdf5
12/12 [==============================] - 1s - loss: 0.0101 - mean_squared_error: 0.0101 - val_loss: 0.0026 - val_mean_squared_error: 0.0026
Epoch 3/10
Epoch 00002: loss improved from 0.01015 to 0.01000, saving model to weights-improvement-02-0.00.hdf5
12/12 [==============================] - 1s - loss: 0.0100 - mean_squared_error: 0.0100 - val_loss: 0.0024 - val_mean_squared_error: 0.0024
Epoch 4/10
Epoch 00003: loss improved from 0.01000 to 0.00977, saving model to weights-improvement-03-0.00.hdf5
12/12 [==============================] - 1s - loss: 0.0098 - mean_squared_error: 0.0098 - val_loss: 0.0021 - val_mean_squared_error: 0.0

In [39]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")

In [40]:
X_val, y_val, w = next(image_label_generator(path_to_folders['train']))

In [41]:
model.predict(X_val)

array([[ 0.04763204],
       [ 0.04611243],
       [ 0.04542437],
       [ 0.04617226],
       [ 0.04782542],
       [ 0.04927502],
       [ 0.05671517],
       [ 0.0418238 ],
       [ 0.04216757],
       [ 0.0431305 ],
       [ 0.04857197],
       [ 0.04199147]], dtype=float32)

In [23]:
y_val

array([-0.09773462,  0.07132844,  0.0617599 ,  0.1670138 ,  0.        ,
        0.        ,  0.0617599 , -0.1167233 , -0.05975719,  0.1670138 ,
        0.        ,  0.1670138 ])